# Домашняя оабота №12

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует  
2. (дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

## mBART huggingface
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210
* Model: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [2]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 25.0 MB/s 
     |████████████████████████████████| 1.3 MB 63.1 MB/s 
     |████████████████████████████████| 182 kB 73.3 MB/s 
     |████████████████████████████████| 7.6 MB 62.8 MB/s 


#### Загружаем модели

In [3]:
from transformers import AutoTokenizer, MBartForConditionalGeneration #AutoModel

model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

In [4]:
print(type(tokenizer), type(model))

<class 'transformers.models.mbart.tokenization_mbart_fast.MBartTokenizerFast'> <class 'transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration'>


#### Суммаризация текста

In [12]:
article_text = """В правительстве Москвы назвали число получивших поддержку города предприятий
Половина крупных и средних предприятий Москвы получила поддержку города в 2022 году
Владимир Ефимов
Владимир Ефимов. Фото: пресс-служба Комплекса экономической политики и имущественно-земельных отношений города Москвы

В 2022 году около 400 крупных и средних промышленных предприятий Москвы получили городскую поддержку. Об этом сообщил заместитель мэра столицы по вопросам экономической политики и имущественно-земельных отношений Владимир Ефимов на заседании Проектного офиса по улучшению инвестиционного климата в Москве. На встрече обсуждались результаты реализации мер поддержки промышленности и инвесторов в 2022 году.

Системная поддержка промышленных предприятий включает в себя более 20 механизмов. В частности, 66 производств имеют статус промкомплекса, который дает им значительные налоговые льготы. Региональная часть ставки налога на прибыль для них снижена с 17 до 13,5 процента, земельный налог ― на 80 процентов, налог на имущество ― на 50 процентов.

В Москве расположено более 3,5 тысячи промышленных площадок, свыше 700 из них ― крупные и средние заводы, отметил вице-мэр.

Предприятиям доступны как системные, так и антикризисные меры городской финансовой и нефинансовой поддержки ― субсидии, целевые и льготные займы, специальные статусы, позволяющие экономить на налогах

Владимир Ефимов
заместитель мэра столицы по вопросам экономической политики и имущественно-земельных отношений
Владимир Ефимов также отметил, что по результатам проделанной правительством Москвы работы к концу года поддержкой в столице охвачено каждое второе крупное и среднее предприятие.

В рамках антикризисной поддержки бизнеса весной 2022 года мэр Москвы Сергей Собянин принял решение автоматически продлить на один год все специальные статусы. Таким образом, предприятия смогут продолжить получать льготы без подтверждения соответствия установленным требованиям.

«В основе введенных в этом году антикризисных мер лежали реальные запросы предприятий города, что и обеспечило такой высокий спрос на новые механизмы. Например, Центр поддержки промышленности получил более 300 обращений по разным вопросам ― от подбора поставщиков до локализации производств», ― почеркнул руководитель Департамента инвестиционной и промышленной политики Москвы Владислав Овчинский. По словам Овчинского, не менее востребована стала финансовая помощь.

10 миллиардов рублей
сумма одобренных предприятиям заявок на льготные инвестиционные кредиты

В рамках антикризисной поддержки с марта 2022 года у предпринимателей, реализующих в Москве масштабные инвестиционные проекты, появилась возможность получать земельные участки в аренду по ставке один рубль в год для расширения существующих и создания новых производств. В настоящее время Москва заключила договоры о выделении земли по льготной ставке с 13 инвесторами.

Работающие в Москве предприятия могут обратиться за помощью в Центр поддержки промышленности по телефону +7(495) 630-00-00 с понедельника по пятницу с 9 до 18 часов или через электронную почту industrysupport@mos.ru. Больше информации об экономической политике и имущественно-земельных отношениях Москвы можно получить на сайте economy.mos.ru."""


input_ids = tokenizer.prepare_seq2seq_batch(
    [article_text],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

output_ids = model.generate(
    input_ids=input_ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [13]:
print(summary)

В 2022 году около 400 крупных и средних промышленных предприятий Москвы получили городскую поддержку. Об этом сообщил заместитель мэра столицы по вопросам экономической политики и имущественно-земельных отношений Владимир Ефимов на заседании Проектного офиса по улучшению инвестиционного климата в Москве. На встрече обсуждались результаты реализации мер поддержки промышленности и инвесторов в 2022 году.


In [14]:
print(model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN